In [1]:
import os, argparse
from build_nnfps import main as build_nnfps_main
import numpy as np
import pandas as pds
import pickle, math
from preprocessing import data_preparation, _char_set, get_property, canonocalize
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import SSVAE
import torch

/root/miniconda3/envs/torchkick/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Molecule fingerprint regeneration for transfer learning, if necessary

In [2]:
# training datafiles to use, pick the combination as you like
csv_files = ['./data/table_2_MP_IE_EA.csv',
             './data/table_2_ZINC_310k.csv'] 
# 'data/paper_clean_DC.csv' is skipped for now because it has many wrong & unphysical labels because of NLP

out_pkl = 'data/smiles2nn.pkl'
if os.path.isfile(out_pkl):
    os.system('rm '+out_pkl)

first = True
for csv_file in csv_files:
    args = argparse.Namespace()
    args.csv_file = csv_file
    if first:
        args.input_vocab_file = ''
        args.output_vocab_file = out_pkl
        args.fp_check = False
        # build_nnfps_main(args)
        first = False
    else:
        args.input_vocab_file = out_pkl
        args.output_vocab_file = out_pkl
        args.fp_check = False
        # build_nnfps_main(args)

# Prepare Combination of Dirty Data for Training SSVAE Model

In [3]:
# are we doing pre-training version? if yes, change pretrain to True
pretrain = False
pretrain_pkl = 'data/smiles2nn.pkl'

# we will try train generative model with these training data files
if 'csv_files' in locals():
    data_uri = csv_files
else:
    data_uri = ['./data/table_2_MP_IE_EA.csv',
                './data/table_2_ZINC_310k.csv'] 
# 'data/paper_clean_DC.csv' is skipped 

# first check how many data points we have in the training data
tmp_smiles = {}
for csv_in in data_uri:
    data = pds.read_csv( csv_in )
    for i in range(len(data['SMILES'])):
        tmp_smiles[ canonocalize(data['SMILES'][i]) ] = True

# define training, validation, and test set division
ntotal = len(tmp_smiles)
ntrn = math.floor( 0.9 * ntotal )
ntst = ntotal - ntrn
frac_val = 0.05
del data, tmp_smiles

# data preparation
if pretrain:
    data, scaler_Y = data_preparation(data_uri, ntrn, ntst,
                                      frac_val = frac_val,
                                      pretrain_uri = pretrain_pkl)
else:
    data, scaler_Y = data_preparation(data_uri, ntrn, ntst,
                                      frac_val = frac_val)

# tag for dumping intermediate models
dt = '221214034018'
model_dir = 'models/'+dt
tmp_model_tag = model_dir+'/'    
scaler_Y_pkl = 'preprocessed_scaler_Y.pkl'
scaler_Y = pickle.load(open(tmp_model_tag+scaler_Y_pkl,'rb'))

print('::: Data preparation completed')

::: Data preparation completed


# Load Previously Trained Model, if Needed

In [4]:
# pre-defined parameters
beta=10000.
char_set = _char_set()
dim_z = 100
dim_h = 250
n_hidden = 3
batch_size = 100

# tag for dumping intermediate models
save_uri = tmp_model_tag+'model_final.pth.tar'

# Instantiate the model
model = SSVAE.TorchModel(sample_data = data, dim_z = dim_z, dim_h = dim_h,
                         n_hidden = n_hidden, batch_size = batch_size, beta = float(beta), char_set = char_set,
                         tmp_model_tag = tmp_model_tag)
dev = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model.initialize(torch.Tensor(data['trnY']), torch.Tensor(data['trnMask']), dev)
model.load_state_dict( torch.load(save_uri)['state_dict'] )
model.eval()

TorchModel(
  (predictor): bi_GRU_zero(
    (GRU): GRU(43, 250, num_layers=3, batch_first=True, bidirectional=True)
    (linear_out_y): Linear(in_features=500, out_features=10, bias=True)
  )
  (encoder): bi_GRU_init(
    (GRU): GRU(86, 250, num_layers=3, batch_first=True, bidirectional=True)
    (linear_in): Linear(in_features=5, out_features=500, bias=True)
    (linear_peek): Linear(in_features=5, out_features=43, bias=True)
    (linear_out_z): Linear(in_features=500, out_features=200, bias=True)
  )
  (decoder): uni_GRU_init(
    (GRU): GRU(86, 250, num_layers=3, batch_first=True)
    (linear_in): Linear(in_features=105, out_features=250, bias=True)
    (linear_peek): Linear(in_features=105, out_features=43, bias=True)
    (linear_out_x): Linear(in_features=250, out_features=43, bias=True)
  )
  (softmax): Softmax(dim=-1)
  (sigmoid): Sigmoid()
)

# Property Prediction Performance

In [5]:
scaled_tstY_hat = model.predict_routine(sample_data = data).detach().cpu().numpy()
tstY_hat = [scaler_Y[i].inverse_transform(scaled_tstY_hat[:,i:i+1]) for i in range(scaled_tstY_hat.shape[1])]
tstY_hat = np.concatenate(tstY_hat, axis=1)

dim_y = data['dim_y']
tstY = data['tstY']
tstMask = data['tstMask']
Y_names = data['Y_names']
for j in range(dim_y):
    idx = np.where( tstMask[:,j] == 1 )[0]
    print('Label Name:', Y_names[j])
    print([j, mean_absolute_error(tstY[idx,j], tstY_hat[idx,j])])

Label Name: EA
[0, 0.11833686830086738]
Label Name: IE
[1, 0.10809764305286138]
Label Name: LogP
[2, 0.047660820839773604]
Label Name: MolWt
[3, 2.3524555872623063]
Label Name: QED
[4, 0.00636211886162179]


# Unconditional Generation

In [6]:
print('Unconditional')
for t in range(10):
    smi = model.sampling_unconditional()
    print([t, smi, get_property(smi)])

Unconditional
[0, 'CNC(=O)C1CCN(C(=O)c2cc3ccccc3[nH]2)CC1', [285.147726848, 1.7660999999999996, 0.8828423886537324]]
[1, 'COc1ccc(OCCOCCOCCOCCO)cc1OC', [330.167853172, 1.1246999999999998, 0.5124030824224903]]
[2, 'O=C(CCCn1c(=O)[nH]c2ccccc2c1=O)NCCc1cccs1', [357.11471246800005, 1.8903999999999999, 0.6772074439180006]]
[3, 'CCc1c(C)[nH]c(SCC(=O)N2CCOCC2)c(=O)c1C#N', 'invalid']


[04:06:30] Can't kekulize mol.  Unkekulized atoms: 2 3 6 19


[4, 'COc1ccc(-c2nnc(SCc3ccccc3)s2)cc1F', [332.04533325600005, 4.645100000000002, 0.6373737411448833]]
[5, 'Cc1cccc(C)c1NC(=O)c1cnc(-c2cccs2)o1', [298.07759868799997, 4.272240000000003, 0.7851852233398187]]
[6, 'CCOC(=O)N1CCN(S(=O)(=O)c2ccc(S(=O)(=O)N3CCC(C(=O)OCC)CC3)cc2)C1', [503.13960688799995, 1.070700000000001, 0.5052175647815739]]
[7, 'O=C(CCc1ccc(S(=O)(=O)N2CCOCC2)cc1)Nc1cccc(C(F)(F)F)c1F', [460.107991, 3.436700000000002, 0.6706119628595901]]
[8, 'COc1ccc(C=Nc2ccc(C#N)cc2)cc1', [236.094963004, 3.3174800000000015, 0.7672341885092581]]
[9, 'CC1CCCC(C)N1C(=O)CSc1nnc(-c2ccccc2[N+](=O)[O-])n1CC', [403.16781066, 3.754800000000003, 0.41408540707345676]]


# Conditional Generation

In [7]:
def print_log(text):
    outfile = 'gen_log.txt'
    if os.path.isfile(outfile):
        open(outfile,'a').write(text+'\n')
    else:
        open(outfile,'w').write(text+'\n')
    return

print('Conditional')
# Determine the property values we want to use for conditional generation
ynames = ['LogP']
yids = [Y_names.index(yname) for yname in ynames]

ytargets = [2.5]
ymeans = np.array([scaler_Y[yid].mean_[0] for yid in yids])
ystds = np.array([np.sqrt(scaler_Y[yid].var_[0]) for yid in yids])
ytargets_transform = ( np.array(ytargets) - ymeans ) / ystds
                        
print(ynames, ':', ytargets)
for t in range(30000):
    smi = model.sampling_conditional(yids, ytargets_transform)
    props = get_property(smi)
    print([t, smi, props])
    print_log(str([t, smi, props]))
print('')

Conditional
['LogP'] : [2.5]
[0, 'Cc1nc(SCC(=O)Nc2ccc3c(c2)OCO3)n2nc(C)cc2n1', [357.0895603400001, 2.20064, 0.7158672526004813]]
[1, 'O=C(c1ccc2c(c1)OCO2)N1CCN(c2nc3ccccc3o2)CC1', [351.121906024, 2.5189000000000004, 0.7067002018652435]]
[2, 'CCn1ncc2c(N3CCN(c4ccccc4F)CC3)nc(-c3ccno3)nc21', [393.17133648000004, 2.9669000000000016, 0.5275365050891538]]
[3, 'CCNC(=O)NC(=O)c1ccc(NC(=O)Nc2ccccc2)cc1', [326.13789043599996, 2.789900000000001, 0.694932360140402]]
[4, 'Cc1ccc(C(=O)NCc2cccs2)cn1', [232.067034004, 2.38152, 0.8824427947003282]]
[5, 'CCOC(=O)c1c(C)[nH]c(C(=O)CN2CCN(Cc3ccc(OC)c(OC)c3)CC2)c1C#N', [454.22162005999996, 2.3892000000000007, 0.45458424974456685]]
[6, 'COC(=O)C1CCN(C(=O)C2CCCN(S(=O)(=O)Cc3ccc(Cl)cc3)C2)CC1', [442.132920644, 2.293400000000001, 0.6530623701938403]]
[7, 'CCN(CC)S(=O)(=O)c1cccc(C(=O)N2CCN(c3ccccc3F)CC2)c1', [419.167890912, 2.818600000000001, 0.7225004274152408]]
[8, 'CC(=O)N1CCC(NC(=O)C2CCN(S(=O)(=O)c3ccc(Br)cc3)CC2)c2ccccc21', [519.082739412, 3.46390000000000

[04:09:32] Can't kekulize mol.  Unkekulized atoms: 4 5 6 10 11 12 14 21 22 23 24


[35, 'Cc1c(C(=O)COC(=O)c2ccc(F)cc2)cccc1C#N', [297.080121464, 3.0455000000000014, 0.6422972247780726]]
[36, 'Cc1cnc(C(=O)N2CCc3ccccc3C2)s1', [258.082684068, 2.6500200000000014, 0.787561574408915]]
[37, 'COc1ccc(S(=O)(=O)Nc2ccc(S(=O)(=O)NCc3ccco3)cc2)cc1', [422.060628296, 2.567500000000001, 0.5765802416870287]]
[38, 'N#Cc1cscc1[N+](=O)[O-]', [153.983698304, 1.52798, 0.45463424677950554]]
[39, 'CCCN1CCN(C(=O)c2cc(-c3ccc(F)cc3)on2)CC1', [317.1539551, 2.648500000000001, 0.8697074378673098]]
[40, 'O=C(OC1CCCCC1)c1ccc([N+](=O)[O-])cc1', [249.10010796, 3.0843000000000016, 0.4686175136722601]]
[41, 'Cc1ccc(C)c(NC(=O)COC(=O)c2n[nH]c3ccccc23)c1', [323.126991404, 2.975240000000001, 0.7230042098599823]]
[42, 'C#C[C@@H]1CS[C@H](C#N)CS1', [169.001991224, 1.36038, 0.5115786589283942]]
[43, 'Cc1nc(-c2ccc(C(=O)N3CCN(C4CCCC4)CC3)cc2)no1', [340.189926008, 2.745420000000001, 0.8596091308697427]]
[44, 'CC(C)(C)OC(=O)N1CCN(C(=O)c2ccccc2[N+](=O)[O-])CC1', [335.14812077199997, 2.2877, 0.6104551476662257]]
[45

[04:10:24] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 24


[48, 'CN(C)S(=O)(=O)N1CCCC(C(=O)N2CCN(c3cccc(C(F)(F)F)c3)CC2)C1', [448.175596384, 1.872400000000001, 0.7057222172808001]]
[49, 'CC1CCCCC1NC(=O)c1cc([N+](=O)[O-])ccc1OC', [292.14230712, 2.911900000000001, 0.6830285591568853]]
[50, 'CCN(CC)C(=O)C(C)Oc1ccc(OC)cc1', [251.152143532, 2.3309000000000006, 0.77859080345729]]
[51, 'O=C1CC(c2ccc(Cl)c(Cl)c2)CN1', [229.006119268, 2.5969000000000007, 0.7887661319604471]]
[52, 'COC(=O)c1c(C)[nH]c(C(=O)COc2cccc(C)c2)n1', [288.111006992, 2.07484, 0.6725718444126026]]
[53, 'COC(=O)c1sccc1S(=O)(=O)Nc1ccc(OC)cc1', [327.02351451600003, 2.3441, 0.8525340551598138]]
[54, 'COc1ccc(C=C2NC(=O)N(CC(C)C)C2=O)cc1OC', [304.14230712, 2.2525000000000004, 0.6690152113196245]]
[55, 'CCN(CC)c1ccc([N+](=O)[O-])cc1[N+](=O)[O-]', [239.090605896, 2.3491999999999997, 0.5806926116844343]]
[56, 'Cc1ccc(-c2cnc(CCC(=O)N3CCC(C(N)=O)CC3)n2)c2ccccc12', 'invalid']


...

[29905, 'N#CC[C@@H]1CC[C@@H]1OC', [125.084063972, 1.32508, 0.5573313081336637]]
[29906, 'CN(Cc1nccn1C)C(=O)Nc1cccc(C(F)(F)F)c1', [312.11979576000004, 3.102800000000001, 0.9465506977215895]]
[29907, 'CC(NC(=O)C1CCN(S(=O)(=O)c2ccc(Cl)cc2)CC1)C(=O)N1CCCCC1C', [455.16455512, 2.6464000000000008, 0.7389325932914379]]
[29908, 'CCN1CCCC1CNC(=O)c1cc(-c2ccc(OC)cc2)on1', [329.173941596, 2.5643000000000002, 0.8820435721213169]]
[29909, 'Cn1ncc2c(C(=O)N3CCCC(C(=O)Nc4ccccn4)CC3)cccc21', [377.18517497600004, 2.8493000000000013, 0.7610955511555553]]
[29910, 'COC(=O)c1nc(-c2ccccc2)cs1', [219.035399528, 2.596700000000001, 0.7286480017582675]]
[29911, 'CC(=O)c1ccc(CN2CCN(Cc3ccc4c(c3)OCCO4)CC2)nc1', [367.18959166, 2.3732000000000006, 0.7565832582823774]]
[29912, 'C=CCC(NC(=O)NCc1ccccn1)C(F)(F)F', [273.108896728, 2.3878000000000004, 0.8092681150246783]]
[29913, 'O=c1c2ccccc2ncn1Cc1nc2ccccc2[nH]1', [276.101111004, 2.3210999999999995, 0.610833273751856]]
[29914, 'N#Cc1cc2ccccc2nc1C(F)F', [204.049904632, 3.04

[14:23:48] Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 25


[29945, 'O=S(=O)(c1cc(F)cc(F)c1)N1CCN(Cc2nc3ccccc3[nH]2)CC1', [392.111853256, 2.347600000000001, 0.7401117772773298]]
[29946, 'O=C1OCCC1C=Cc1cccc([N+](=O)[O-])c1', [233.068807832, 2.1711, 0.4554565538867987]]
[29947, 'CC1CCN(c2ncc(Cl)cn2)CC1', [211.08762512799998, 2.3663, 0.7143813564494012]]
[29948, 'Cc1c(NC(=O)COC(=O)c2ccc([N+](=O)[O-])cc2)c(=O)n(-c2ccc3c(c2)OCO3)s1', [457.05798544, 2.639820000000001, 0.338557782722782]]
[29949, 'C=Cc1cc(Cl)ccc1C(=O)O', [182.013457144, 2.6812000000000005, 0.7637502769614091]]
[29950, 'C#Cc1c(OCCOCCOC)ccc2c1NSN2', [280.088163372, 2.1103, 0.4531185615188521]]
[29951, 'COc1ccc(C(=O)c2ccccn2)cc1', [213.078978592, 2.3212, 0.7343399476261179]]
[29952, 'CCc1ccc2c(c1)C(=O)c1ccccc1C2=O', [236.083729624, 3.024400000000001, 0.6504723859039159]]
[29953, 'c1ccc2c(c1)ncn2Cc1cccc(-n2cncn2)c1', [275.117095416, 2.6653000000000002, 0.577286667826587]]
[29954, 'CC1CC(C)CN(S(=O)(=O)c2cccc(C(=O)OCC(=O)c3ccco3)c2)C1=O', [419.10387301199995, 2.512500000000001, 0.5223709702

[14:27:42] Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 25


[29999, 'CCOC(=O)CSc1nnc(-c2ccc3c(c2)OCO3)s1', [324.023848864, 2.5890000000000004, 0.618137954458981]]

